# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [59]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [51]:
#corrected-data_path = 'weatherAPI_data.csv'
corrected_data_path = "../output_data/cities.csv"

corrected_data_df = pd.read_csv(corrected_data_path)
corrected_data_df


,City_ID,City_ID.1,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,2264557,Ponta do Sol,75,PT,1603744595,72,32.67,-17.10,66.20,8.05
1,1,501847,Rochegda,97,RU,1603744596,97,62.80,43.25,28.35,9.24
2,2,5880054,Barrow,90,US,1603744597,85,71.29,-156.79,19.40,5.82
3,3,3366880,Hermanus,85,ZA,1603744598,85,-34.42,19.23,63.00,4.00
4,4,1106677,Bambous Virieux,40,MU,1603744599,73,-20.34,57.76,73.40,10.29
5,5,3430443,Necochea,0,AR,1603744600,60,-38.55,-58.74,60.87,5.97
6,6,2136825,Isangel,18,VU,1603744604,72,-19.55,169.27,77.20,13.73
7,7,4030556,Rikitea,100,PF,1603744605,84,-23.12,-134.97,75.36,18.95
8,8,5866583,Saint Paul Harbor,40,US,1603744606,46,57.79,-152.41,50.00,16.11
9,9,2965535,Cavan,41,IE,1603744607,87,53.99,-7.36,45.00,3.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [52]:
gmaps.configure(api_key=g_key)
locations = corrected_data_df[["Lat","Lng"]]
humidity = corrected_data_df["Humidity"]

In [53]:
fig = gmaps.figure(center=(50.0,-5.0), zoom_level=1)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [63]:
#humidity below 60 and wind speed below 5  this is not working??????
new_df = corrected_data_df[(corrected_data_df["Cloudiness"] == 0) & (corrected_data_df["Wind Speed"] <= 70) & (corrected_data_df["Humidity"] <= 100)].dropna()
new_df


,City_ID,City_ID.1,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
5,5,3430443,Necochea,0,AR,1603744600,60,-38.55,-58.74,60.87,5.97
22,22,2447513,Arlit,0,NE,1603744621,15,18.74,7.39,80.24,9.93
25,25,2074865,Carnarvon,0,AU,1603744624,82,-24.87,113.63,68.00,19.46
28,28,2126710,Beringovskiy,0,RU,1603744627,91,63.05,179.32,31.66,2.93
42,42,3129371,Arroyo de la Encomienda,0,ES,1603744644,87,41.61,-4.80,48.00,11.41


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [64]:
hotel_df = new_df.loc[:,["City_ID", "City", "Country", "Lat", "Lng"]]
hotel_df
hotel_df["Hotel_Name"]=""
hotel_df

,City_ID,City,Country,Lat,Lng,Hotel_Name
5,5,Necochea,AR,-38.55,-58.74,
22,22,Arlit,NE,18.74,7.39,
25,25,Carnarvon,AU,-24.87,113.63,
28,28,Beringovskiy,RU,63.05,179.32,
42,42,Arroyo de la Encomienda,ES,41.61,-4.80,


In [73]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000 , 
    "keyword": "hotel",
    "types": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    time.sleep(1)    
    # get hotel location
    lat = row['Lat']
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params['location'] = f"{lat},{lng}"

    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    #print(response)
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel_Name'] = results[0]['name']
       # print(results[0]['name'])
        #types_df.loc[index, 'address'] = results[0]['vicinity']
        #types_df.loc[index, 'price_level'] = results[0]['price_level']
        #types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Closest hotel in Necochea is Gran Hotel Avenida.
------------
Closest hotel in Arlit is Hôtel Telwa Bungalow.
------------
Closest hotel in Carnarvon is The Gascoyne Hotel.
------------
Closest hotel in Beringovskiy is Beringgorski.
------------
Closest hotel in Arroyo de la Encomienda is AC Hotel by Marriott Palacio de Santa Ana.
------------


In [74]:
hotel_df

,City_ID,City,Country,Lat,Lng,Hotel_Name
5,5,Necochea,AR,-38.55,-58.74,Gran Hotel Avenida
22,22,Arlit,NE,18.74,7.39,Hôtel Telwa Bungalow
25,25,Carnarvon,AU,-24.87,113.63,The Gascoyne Hotel
28,28,Beringovskiy,RU,63.05,179.32,Beringgorski
42,42,Arroyo de la Encomienda,ES,41.61,-4.80,AC Hotel by Marriott Palacio de Santa Ana


In [76]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [77]:
# Add marker layer ontop of heat map
markerlayer=gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markerlayer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))